In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
movies = pd.read_csv('dataset/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(2)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [2]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [7]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count','popularity','keywords', 'overview']]
movies_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [4]:
movies_df.genres

0       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1       [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3       [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
                              ...                        
4798    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4799    [{"id": 35, "name": "Comedy"}, {"id": 10749, "...
4800    [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...
4801                                                   []
4802                  [{"id": 99, "name": "Documentary"}]
Name: genres, Length: 4803, dtype: object

In [8]:
# genres 칼럼은 문자열로 표기되어 있으며 문자열을 분해해서 개별 장르를 파이썬 리스트 객체로 추출 필요
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
movies_df['genres'][:1]

0    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
Name: genres, dtype: object

In [9]:
movies_df['genres'] = movies_df['genres'].apply(lambda x:[y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."


In [10]:
df5 = movies_df[['genres','keywords']]
df5['genres'].apply(lambda x:(' ').join(x)).head()

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
2                      Action Adventure Crime
3                 Action Crime Drama Thriller
4            Action Adventure Science Fiction
Name: genres, dtype: object

[과제] 아래사항을 수행하세요
- 장르 콘텐츠 필터링을 이용한 영화 추천(count 방식으로 feature 벡터화, cosine_similarity로 유사도 측정)
- 평점에 평가 횟수를 반영하여 새롭게 평점 기준을 정의한 후 적용하여 영화 추천(vote_count를 포함하여 아래 가중평점 사용)
 - m값은 평점 투표 횟수가 많은 영화에 더 많은 가중 평점을 부여
 - m값은 전체 투표 횟수에서 상위 60%에 해당하는 횟수를 기준으로 정함
 - 가중평점 : (v/(v+m))*R + (m/(v+m))*C
   - v : 개별 영화에 평점을 투표한 횟수 
   - m : 평점을 부여하기 위한 최소 투표 횟수
   - R : 개별 영화에 대한 평균 평점
   - C : 전체 영화에 대한 평균 평점


##### 데이터 로드 및 전처리

In [3]:
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
movies = pd.read_csv('./dataset/tmdb_5000_movies.csv', encoding='utf-8')
# 컨텐츠를 추천하는 시스템을 만들기 위해서
# 해당 영화들의 유사도 비교할 항목들만 추출
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                 'popularity', 'keywords', 'overview']]

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
# genres라는 dict형태에서 'name' key의 value들만 가져오기
movies_df['genres'] = movies_df['genres'].apply(lambda x: [y['name'] for y in x])
# keywords도 동일하게 수행
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [y['name'] for y in x])
# 리스트에 담겨있는 genres, keywords를 공백 기준으로 문자열로 붙이기
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df['keywords_literal'] = movies_df['keywords'].apply(lambda x: (' ').join(x))

##### 텍스트 데이터를 Feature Vectorization

In [4]:
# 모든 변수가 텍스트로 이루어졌으니 Countvectorizer로 feature 벡터화시키기
# 경우에 따라 추가적으로 Lemmatization 추가해 토큰화 전용 함수 추가도 가능!

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
# fit_transform안에 데이터프레임형태로 넣어주면 안됨. 하나의 변수씩만 넣어주자!
genres_vect = cnt_vect.fit_transform(movies_df['genres_literal'])
keywords_vect = cnt_vect.fit_transform(movies_df['keywords_literal'])

# 장르에 따른 영화별 코사인 유사도 추출
genre_sim = cosine_similarity(genres_vect, genres_vect)
# 3개만 유사도행렬값 추출해보기
print(genre_sim[:3])

[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]]


##### 유사도 행렬을 기준으로 어벤져스영화와 유사한 10개의 영화가 무엇인지 확인

In [10]:
# argsort를 이용해서 유사도가 높은 영화들의 index 추출
genre_sim_idx = genre_sim.argsort()[::-1]

# 특정 영화를 기준으로 선정해서 유사도가 높은 영화를 반환하는 함수 생성
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    # 비교기준으로 선정할 특정 영화 선정
    title_movie = df[df['title'] == title_name]
    
    # 비교기준 영화의 index.values를 할당해 유사도 행렬에서 비교기준 영화에 해당하는 유사도 행렬값을 찾자!
    title_movie_idx = title_movie.index.values
    # 모든 영화끼리의 유사도 행렬에서 비교기준 영화에 대한 유사도행렬 찾아서 할당
    top_sim_idx = sorted_idx[title_movie_idx, :top_n]
    print(top_sim_idx)
    
    # top_sim_idx는 2차원 array이기 때문에 1차원 array로 변경
    # 왜냐하면 top_sim_idx로 movies_df의 index값으로 넣어서 유사한 영화추출할 것이기 때문
    top_sim_idx = top_sim_idx.reshape(-1,)
    similar_movie = df.iloc[top_sim_idx]
    
    return similar_movie
    
similar_movies = find_sim_movie(movies_df, genre_sim_idx, 'The Avengers')
print(similar_movies[['title','vote_average','vote_count']])
similar_movies.head()

[[   0 2743 2749 2750 2751 2752 2755 2757 2760 2764]]
                      title  vote_average  vote_count
0                    Avatar           7.2       11800
2743   The Butterfly Effect           7.3        2060
2749         Child's Play 2           5.8         308
2750           No Good Deed           5.6         181
2751               The Mist           6.7        1399
2752             Ex Machina           7.6        4737
2755          Earth to Echo           5.7         290
2757  Letters from Iwo Jima           7.2         541
2760                   Room           8.1        2757
2764            Light It Up           6.6           7


,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,keywords_literal,weighted_vote
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,7.166301
2743,1954,The Butterfly Effect,"[Science Fiction, Thriller]",7.3,2060,73.307513,"[amnesia, chaos theory, blackout, time travel,...",A young man struggles to access sublimated chi...,Science Fiction Thriller,amnesia chaos theory blackout time travel flas...,7.116008
2749,11186,Child's Play 2,"[Drama, Horror]",5.8,308,16.681567,"[faithlessness, puppet, killer toys, toy comes...",Chuckie's back as the doll possessed by the so...,Drama Horror,faithlessness puppet killer toys toy comes to ...,5.959484
2750,136835,No Good Deed,"[Crime, Thriller]",5.6,181,24.656068,"[sadistic, hostage, sociopath, serial killer, ...","Terri is a devoted wife and mother of two, liv...",Crime Thriller,sadistic hostage sociopath serial killer blood...,5.930555
2751,5876,The Mist,"[Science Fiction, Horror, Thriller]",6.7,1399,37.104353,"[father son relationship, monster, supermarket...","After a violent storm, a dense cloud of mist e...",Science Fiction Horror Thriller,father son relationship monster supermarket fi...,6.572814


하지만 문제점이 하나 생겼다. Light it Up의 영화는 영화 구성 내용만으로는 어벤져스영화와 매우 유사하긴 하지만 vote_average(평점)에 비해 vote_count(리뷰 개수)가 너무 적다는 것이다. 만약 이 영화를 어벤져스를 본 고객에게 "당신이 좋아할만한 다른 영화 입니다." 라고 추천해준다면 고객이 만족스러워할까? 시나리오 측면에서만 두 개의 영화가 비슷할지라도 흥미를 느낀다는 점에서는 매우 상이한 느낌을 받을 것이다. 따라서 우리는 추가적인 변수로 앞서 배웠던 '가중 평점' 변수를 추가해서 다시 유사한 영화들을 재정렬시켜보자.

In [11]:
C = movies_df['vote_average'].mean()

# 투표횟수 중 60%이상의 횟수에 달하는 숫자
# 예를들어 총 투표횟수가 100과 1일때 m값은 매우 달라진다.
m = movies_df['vote_count'].quantile(0.6)

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R) + ( (m/(m+v)) * C)
    
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
 
# 가중 평점 변수를 추가해서 유사한 영화 찾아주는 함수 재정의
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_idx = title_movie.index.values
    
    similar_idx = sorted_idx[title_idx, :(top_n*2)]
    similar_idx = similar_idx.reshape(-1,)
    
    #자기 자신 영화 제외, boolean index기법 사용!
    similar_idx = similar_idx[similar_idx != title_idx]
    return df.iloc[similar_idx].sort_values(by=['weighted_vote'], ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_idx,
                               'The Avengers')
print(similar_movies[['title','vote_average','weighted_vote']])
similar_movies.head()

                      title  vote_average  weighted_vote
2760                   Room           8.1       7.862312
2752             Ex Machina           7.6       7.490704
0                    Avatar           7.2       7.166301
2743   The Butterfly Effect           7.3       7.116008
2757  Letters from Iwo Jima           7.2       6.749914
2751               The Mist           6.7       6.572814
2783               The Call           6.6       6.484465
2787         Quest for Fire           7.1       6.321415
2742           The Shallows           6.2       6.179394
2778               Infamous           6.4       6.135104


,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,keywords_literal,weighted_vote
2760,264644,Room,"[Drama, Thriller]",8.1,2757,66.113340,"[based on novel, carpet, isolation, kidnapping...",Jack is a young boy of 5 years old who has liv...,Drama Thriller,based on novel carpet isolation kidnapping imp...,7.862312
2752,264660,Ex Machina,"[Drama, Science Fiction]",7.6,4737,95.130041,"[dancing, artificial intelligence, distrust, i...","Caleb, a 26 year old coder at the world's larg...",Drama Science Fiction,dancing artificial intelligence distrust isola...,7.490704
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,7.166301
2743,1954,The Butterfly Effect,"[Science Fiction, Thriller]",7.3,2060,73.307513,"[amnesia, chaos theory, blackout, time travel,...",A young man struggles to access sublimated chi...,Science Fiction Thriller,amnesia chaos theory blackout time travel flas...,7.116008
2757,1251,Letters from Iwo Jima,"[Action, Adventure, Drama, War]",7.2,541,24.472480,"[world war ii, cave, dying and death, pacific ...",The story of the battle of Iwo Jima between th...,Action Adventure Drama War,world war ii cave dying and death pacific thea...,6.749914
